# Hierarchical Indices for RAG

In this notebook, I implement a hierarchical indexing approach for RAG systems. This technique improves retrieval by using a two-tier search method: first identifying relevant document sections through summaries, then retrieving specific details from those sections.

Traditional RAG approaches treat all text chunks equally, which can lead to:

- Lost context when chunks are too small
- Irrelevant results when the document collection is large
- Inefficient searches across the entire corpus

Hierarchical retrieval solves these problems by:

- Creating concise summaries for larger document sections
- First searching these summaries to identify relevant sections
- Then retrieving detailed information only from those sections
- Maintaining context while preserving specific details

## Setting Up the Environment
We begin by importing necessary libraries.

In [21]:

# === Setup & Helpers ===
import os
import re
import pickle
from typing import List, Dict, Any, Tuple, Optional, Union, Callable

import numpy as np

# If you plan to use PDF extraction here:
try:
    from PyPDF2 import PdfReader
except Exception as e:
    print("PyPDF2 not available; please install it if you need PDF text extraction.")
    PdfReader = None

# Gemini setup (correct usage: choose model at construction time)
try:
    import google.generativeai as genai
    GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
    if GEMINI_API_KEY:
        genai.configure(api_key=GEMINI_API_KEY)
except Exception as e:
    print("google-generativeai not available; Gemini calls will fail unless installed and key set.")
    genai = None

def gemini_text(prompt: str, model_name: str = "gemini-1.5-pro") -> str:
    """Correct Gemini usage: pick model in constructor, no model= kwarg in generate_content()."""
    if genai is None:
        return "[Gemini not configured] " + prompt[:200]
    model = genai.GenerativeModel(model_name)
    resp = model.generate_content(prompt)
    return getattr(resp, "text", "").strip()


In [22]:

# === Embeddings (stub) ===
# Replace this with your real embedder (OpenAI, Gemini Embeddings, etc.).
def create_embeddings(texts, model: str = None):
    """Accepts str or List[str]; returns List[np.ndarray] with fixed dim."""
    if isinstance(texts, str):
        texts = [texts]
    dim = 768
    rng = np.random.default_rng(42)  # deterministic demo
    return [rng.random(dim, dtype=float) for _ in texts]


## Simple Vector Store Implementation

In [23]:

# === Vector Store ===
class SimpleVectorStore:
    def __init__(self):
        self.embeddings: List[np.ndarray] = []
        self.documents: List[Dict[str, Any]] = []  # each: {"text": str, "metadata": {...}}

    def _as_unit_vector(self, x):
        v = np.asarray(x, dtype=float).reshape(-1)
        n = np.linalg.norm(v)
        return v if n == 0 else v / n

    def add_item(self, *, text: str, embedding: Union[np.ndarray, List[float]], metadata: Optional[Dict[str, Any]] = None):
        emb = self._as_unit_vector(embedding)
        self.embeddings.append(emb)
        self.documents.append({"text": text, "metadata": (metadata or {})})

    def similarity_search(self, query_embedding, k: int = 5, filter_func: Optional[Callable[[Dict[str, Any]], bool]] = None, return_scores: bool = False):
        if not self.embeddings:
            return [] if not return_scores else []
        q = self._as_unit_vector(query_embedding)
        sims = []
        for i, emb in enumerate(self.embeddings):
            doc = self.documents[i]
            if filter_func is not None and not filter_func(doc):
                continue
            sim = float(np.dot(q, emb))   # cosine because normalized
            sims.append((i, sim))
        if not sims:
            return [] if not return_scores else []
        sims.sort(key=lambda t: t[1], reverse=True)
        top = sims[:max(1, int(k))]
        return [(self.documents[i], score) for i, score in top] if return_scores else [self.documents[i] for i, _ in top]


## Document Processing Functions

In [24]:

# === PDF Processing ===
def extract_text_pages(pdf_path: str) -> List[str]:
    if PdfReader is None:
        raise RuntimeError("PyPDF2 is not installed. Please install it to extract PDF text.")
    pages = []
    with open(pdf_path, "rb") as f:
        reader = PdfReader(f)
        for p in reader.pages:
            pages.append(p.extract_text() or "")
    return pages

def chunk_text(text: str, chunk_size=1000, overlap=200) -> List[Tuple[str, int]]:
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(n, start + chunk_size)
        chunks.append((text[start:end], start))
        if end == n:
            break
        start = max(end - overlap, start + 1)
    return chunks

def process_document(pdf_path: str, chunk_size=1000, chunk_overlap=200) -> SimpleVectorStore:
    print(f"Extracting text from {pdf_path}...")
    pages = extract_text_pages(pdf_path)
    store = SimpleVectorStore()
    all_chunks = []
    for page_idx, page_text in enumerate(pages):
        page_text = page_text.strip()
        if not page_text:
            continue
        for chunk, start_idx in chunk_text(page_text, chunk_size, chunk_overlap):
            all_chunks.append({"text": chunk, "metadata": {"page": page_idx, "char_start": start_idx}})
    print(f"Created {len(all_chunks)} text chunks")
    print("Creating embeddings for chunks...")
    chunk_embeddings = create_embeddings([c["text"] for c in all_chunks], model=None)
    for i, c in enumerate(all_chunks):
        store.add_item(text=c["text"], embedding=chunk_embeddings[i], metadata=c["metadata"])
    print(f"Vector store created with {len(store.documents)} chunks\n")
    return store


In [25]:

# === Generate Response (robust to strings or list of docs) ===
def _normalize_sources(sources):
    out, seen = [], set()
    for s in (sources or []):
        if isinstance(s, str):
            v = s.strip()
        elif isinstance(s, dict):
            title = s.get("title")
            page = s.get("page") or (s.get("metadata") or {}).get("page")
            url  = s.get("url")
            bits = [b for b in [title, f"p.{page}" if page is not None else None, url] if b]
            v = " ".join(bits) if bits else "source"
        else:
            v = str(s)
        if v and v not in seen:
            seen.add(v)
            out.append(v)
    return out

def generate_response(query: str, retrieved_docs, sources: list = None, llm_call=None, max_ctx_chars: int = 1500):
    # Build context
    if isinstance(retrieved_docs, str):
        context = retrieved_docs[:max_ctx_chars]
    else:
        context = ""
        for d in (retrieved_docs or []):
            t = d.get("text", "") if isinstance(d, dict) else str(d)
            if not t:
                continue
            space_left = max_ctx_chars - len(context)
            if space_left <= 0:
                break
            context += t[:space_left] + "\n"

    # Infer sources if not provided
    if sources is None and not isinstance(retrieved_docs, str):
        inferred = []
        for d in (retrieved_docs or []):
            if isinstance(d, dict):
                meta = d.get("metadata") or {}
                page = meta.get("page")
                if page is not None:
                    inferred.append(f"p.{page}")
        sources = inferred

    prompt = f"""Answer the user's question using ONLY the context.

Question:
{query}

Context:
{context.strip()}

If the context is thin, say what's missing and answer conservatively. End with a 'Sources:' line when available.
"""

    text = (llm_call(prompt).strip() if llm_call else "Here’s what the document indicates:\n" + context.strip())
    src_list = _normalize_sources(sources)
    if src_list:
        text += "\n\nSources: " + ", ".join(src_list)
    return text


## Creating Embeddings

In [26]:

# === Simple CRAG & Standard RAG (demo versions) ===
def rag_retrieve(store: SimpleVectorStore, query: str, k: int = 8):
    q_emb = create_embeddings(query)[0]
    return store.similarity_search(q_emb, k=k, return_scores=False)

def crag_retrieve(store: SimpleVectorStore, query: str, k_initial: int = 12, k_final: int = 6):
    # initial retrieve
    q_emb = create_embeddings(query)[0]
    initial_docs = store.similarity_search(q_emb, k=k_initial, return_scores=True)

    # naive re-rank: prefer shorter chunks slightly (toy heuristic)
    rescored = []
    for doc, score in initial_docs:
        length_penalty = 0.05 * (len(doc.get("text", "")) / 1000.0)
        rescored.append((doc, score - length_penalty))
    rescored.sort(key=lambda t: t[1], reverse=True)
    return [d for d, _ in rescored[:k_final]]


In [27]:

# === Grading & Comparison helpers (Gemini) ===
def grade_answer(query: str, answer: str, reference: str = "") -> str:
    prompt = f"""You are grading an answer.

Question:
{query}

Answer:
{answer}

Reference (optional):
{reference}

Give a brief justification and a 0-100 score as 'Score: NN'.
"""
    return gemini_text(prompt)

def compare_responses(query: str, crag_answer: str, rag_answer: str) -> str:
    prompt = f"""Compare two answers and decide which is better.

Question:
{query}

Answer A (CRAG):
{crag_answer}

Answer B (Standard RAG):
{rag_answer}

Assess correctness, grounding, completeness, and clarity.
Return rationale and a final line exactly: 'Winner: A', 'Winner: B', or 'Winner: Tie'.
"""
    return gemini_text(prompt)

def _extract_score(text: str) -> float:
    m = re.search(r"Score:\s*([0-9]+(?:\.[0-9]+)?)", text or "", flags=re.I)
    return float(m.group(1)) if m else 0.0

def _extract_winner(text: str) -> str:
    t = (text or "").strip().lower()
    if "winner: a" in t:
        return "A"
    if "winner: b" in t:
        return "B"
    return "Tie"


In [28]:

# === Full evaluation runner ===
def run_crag_evaluation(pdf_path: str, test_queries: List[str], reference_answers: Optional[List[str]] = None):
    store = process_document(pdf_path)

    results = []
    for i, query in enumerate(test_queries):
        print(f"\n===== Evaluating Query {i+1}/{len(test_queries)} =====\nQuery: {query}\n")

        reference = reference_answers[i] if reference_answers and i < len(reference_answers) else ""

        print("=== Running CRAG ===")
        crag_docs = crag_retrieve(store, query)
        crag_answer = generate_response(query, crag_docs, llm_call=lambda p: gemini_text(p, "gemini-1.5-pro"))

        print("\n=== Running standard RAG ===")
        rag_docs = rag_retrieve(store, query)
        rag_answer = generate_response(query, rag_docs, llm_call=lambda p: gemini_text(p, "gemini-1.5-pro"))

        print("\n=== Evaluating CRAG response ===")
        crag_grade = grade_answer(query, crag_answer, reference)
        print(crag_grade)

        print("\n=== Evaluating standard RAG response ===")
        rag_grade = grade_answer(query, rag_answer, reference)
        print(rag_grade)

        print("\n=== Comparing approaches ===")
        cmp_txt = compare_responses(query, crag_answer, rag_answer)
        print(cmp_txt)

        result = {
            "query": query,
            "crag_answer": crag_answer,
            "rag_answer": rag_answer,
            "crag_score": _extract_score(crag_grade),
            "rag_score": _extract_score(rag_grade),
            "winner": _extract_winner(cmp_txt)
        }
        results.append(result)

    # Summarize
    avg_crag = float(np.mean([r["crag_score"] for r in results])) if results else 0.0
    avg_rag  = float(np.mean([r["rag_score"]  for r in results])) if results else 0.0
    verdict  = "CRAG" if avg_crag > avg_rag else ("Standard RAG" if avg_rag > avg_crag else "Tie")

    overall_analysis = f"Avg(CRAG)={avg_crag:.3f}, Avg(RAG)={avg_rag:.3f} → {verdict}"
    return {"per_query": results, "overall_analysis": overall_analysis}


## Evaluation of Hierarchical and Standard RAG Approaches

In [10]:
# --- PATCH: define create_embeddings if missing ---
import numpy as np

def create_embeddings(texts, model: str = None):
    """
    Accepts a string or list of strings and returns a list of 1D numpy arrays.
    Keep the 'model' param for compatibility with upstream calls.
    Replace this with your real embedder when ready.
    """
    if isinstance(texts, str):
        texts = [texts]
    dim = 768
    rng = np.random.default_rng(42)  # deterministic for testing
    return [rng.random(dim, dtype=float) for _ in texts]


In [11]:
evaluation_results = run_crag_evaluation(pdf_path, test_queries, reference_answers)
print("\n=== Overall Analysis of CRAG vs Standard RAG ===")
print(evaluation_results["overall_analysis"])


Extracting text from /Users/kekunkoya/Desktop/770 Google /AI_Information.pdf...
Created 46 text chunks
Creating embeddings for chunks...
Vector store created with 46 chunks


===== Evaluating Query 1/1 =====
Query: How does machine learning differ from traditional programming?

=== Running CRAG ===

=== Running standard RAG ===

=== Evaluating CRAG response ===
Justification: The answer admits to not answering the question and instead points out what information is missing from the provided source.  It does not attempt to answer the question based on general knowledge.

Score: 0

=== Evaluating standard RAG response ===
Justification: The answer claims the provided source doesn't contain relevant information and fails to answer the question.  It even lists specific pages as sources, implying they were consulted, but then states they lack the necessary information.  This suggests either the sources *were* relevant but the answerer didn't understand them, or the answerer didn't actually 

In [12]:

# --- Demo ---
pdf_path = "/Users/kekunkoya/Desktop/770 Google /AI_Information.pdf"

test_queries = [
    "How does machine learning differ from traditional programming?",
]

reference_answers = [
    "Machine learning differs from traditional programming by having computers learn patterns from data rather than following explicit instructions. In traditional programming, developers write rules; in ML, models learn from examples."
]

evaluation_results = run_crag_evaluation(pdf_path, test_queries, reference_answers)
print("\n=== Overall Analysis of CRAG vs Standard RAG ===")
print(evaluation_results["overall_analysis"])


Extracting text from /Users/kekunkoya/Desktop/770 Google /AI_Information.pdf...
Created 46 text chunks
Creating embeddings for chunks...
Vector store created with 46 chunks


===== Evaluating Query 1/1 =====
Query: How does machine learning differ from traditional programming?

=== Running CRAG ===

=== Running standard RAG ===

=== Evaluating CRAG response ===
Justification: The answer admits it doesn't address the question and the provided "sources" are meaningless without context.  While the optional reference provides the correct distinction, the answer itself fails to convey this information.

Score: 0

=== Evaluating standard RAG response ===
Justification: The answer acknowledges the inability to answer the question based on the provided document and correctly states that more information is needed. However, it lists source page numbers that are irrelevant and do not seem to connect to the non-existent explanation.  While acknowledging limitations is good, citing irrelevant sour